In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import random as r
import seaborn as sns
import matplotlib.pyplot as plt

from time import time
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, GridSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, classification_report

In [2]:
class CFG(object):
    def __init__(self,
         seed: int = 42,
         n_splits: int = 5,
         show_info: bool = False,
         ):

        self.seed = seed
        self.n_splits = n_splits
        self.show_info = show_info
        self.tr_data = "../input/heart-disease-prediction-tfug-chd-oct-2022/train.csv"
        self.ss_data = "../input/heart-disease-prediction-tfug-chd-oct-2022/sample_submission.csv"
        self.ts_data = "../input/heart-disease-prediction-tfug-chd-oct-2022/test.csv"
        self.model_save_path = "models"
        if not os.path.exists(self.model_save_path): os.makedirs(self.model_save_path)

cfg = CFG(seed=42, show_info=True)

In [3]:
def breaker(num: int=50, char: str="*") -> None:
    print("\n" + num*char + "\n")
    

def print_scores(accuracy: float, auc: float, precision: np.ndarray, recall: np.ndarray, f_score: np.ndarray) -> None:
    print(f"Accuracy  : {accuracy:.5f}")
    print(f"ROC-AUC   : {auc:.5f}")
    print(f"Precision : {precision}")
    print(f"Recall    : {recall}")
    print(f"F-Score   : {f_score}")
    

def get_scores(y_true: np.ndarray, y_pred: np.ndarray) -> tuple:
    accuracy = accuracy_score(y_pred, y_true)
    auc = roc_auc_score(y_pred, y_true)
    precision, recall, f_score, _ = precision_recall_fscore_support(y_pred, y_true)

    return accuracy, auc, precision, recall, f_score

In [4]:
tr_df = pd.read_csv(cfg.tr_data)
tr_df = tr_df.drop(columns=["index"])

X = tr_df.iloc[:, :-1].copy().values
y = tr_df.iloc[:, -1].copy().values

sc_X = StandardScaler()
X = sc_X.fit_transform(X)

if cfg.show_info:
    breaker()
    for val in set(y):
        print(f"Class {val} count : {y[y == val].shape[0]}")
breaker()

        
parameters = {
    "n_estimators" : [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000],
}


**************************************************

Class 0 count : 386
Class 1 count : 454

**************************************************



In [5]:
# model_acc = GridSearchCV(
#     RandomForestClassifier(random_state=cfg.seed),
#     parameters,
#     cv=5,
#     scoring="accuracy"
# )
# model_acc.fit(X, y)

# breaker()
# print(f"Best Params : {model_acc.best_params_}")
# print(f"Best Score  : {model_acc.best_score_}")

# breaker()

In [6]:
model_auc = GridSearchCV(
    RandomForestClassifier(random_state=cfg.seed),
    parameters,
    cv=5,
    scoring="neg_log_loss"
)
model_auc.fit(X, y)

breaker()
print(f"Best Params : {model_auc.best_params_}")
print(f"Best Score  : {model_auc.best_score_}")

breaker()


**************************************************

Best Params : {'n_estimators': 500}
Best Score  : -0.7041383944760992

**************************************************



In [7]:
ts_df = pd.read_csv(cfg.ts_data)
ts_df = ts_df.drop(columns=["index"])
X_test = ts_df.copy().values

X_test = sc_X.transform(X_test)
y_pred = model_auc.predict_proba(X_test)[:, 1]

ss_df = pd.read_csv(cfg.ss_data)
ss_df["target"] = y_pred
ss_df.to_csv("submission.csv", index=False)